# Assignment - 05
#### Read Ireland Population data from 

https://data.cso.ie/#
#### In that go to pupulation FY006A
#### Get API Data Query Link to read data in csv
https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en
#### Read data in to CSV from link using pands
#### Dope a column using pands refer document
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html

In [121]:
# import requir library
import pandas as pd
import numpy as np

In [122]:
# read the csv file from the URL
df = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en')
print(df.head(3))

   STATISTIC Statistic Label  TLIST(A1)  CensusYear C02199V02655         Sex  \
0  FY006AC01      Population       2022        2022            -  Both sexes   
1  FY006AC01      Population       2022        2022            -  Both sexes   
2  FY006AC01      Population       2022        2022            -  Both sexes   

  C02076V03371 Single Year of Age                          C03789V04537  \
0            -           All ages                                   IE0   
1            -           All ages  2ae19629-1492-13a3-e055-000000000001   
2            -           All ages  2ae19629-1433-13a3-e055-000000000001   

  Administrative Counties    UNIT    VALUE  
0                 Ireland  Number  5149139  
1   Carlow County Council  Number    61968  
2     Dublin City Council  Number   592713  


In [123]:
# Save to local file
FileName = 'population.csv'
Location = '../data/'

In [124]:
# Save the dataframe to a CSV file
# This CSV file save to check data locally
df.to_csv(Location + FileName, index=False) 

In [125]:
# Manuplate the data as needed
# Remove unwanted columns
df.drop(columns=['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'C02199V02655', 'C02076V03371', 'C03789V04537', 'UNIT'], inplace=True )
# Filter out rows where 'Single Year of Age' is 'All Ages'
df = df[df['Single Year of Age'] != 'All ages']
df = df[df['Administrative Counties'] != 'Ireland']
df = df[df['Sex'] != 'Both sexes']
# Replace some string of age in to valve
df['Single Year of Age'] = df['Single Year of Age'].replace({'Under 1 year': '0', '100 years and over': '100'})
# Remove non-digit characters and convert to integer
df['Single Year of Age'] = df['Single Year of Age'].replace('\D', '', regex=True)
df['Single Year of Age']=df['Single Year of Age'].astype('int64')

<>:11: SyntaxWarning: invalid escape sequence '\D'
<>:11: SyntaxWarning: invalid escape sequence '\D'
/tmp/ipykernel_2715/2231240295.py:11: SyntaxWarning: invalid escape sequence '\D'
  df['Single Year of Age'] = df['Single Year of Age'].replace('\D', '', regex=True)


In [126]:
# Save the cleaned dataframe to a CSV file for further analysis
df.to_csv(Location + FileName, index=False) 

In [127]:
# Conver data in to the pivot table for better analysis
df_pivot = pd.pivot_table(df, columns=['Sex', 'Single Year of Age'], index=['Administrative Counties'], values ='VALUE')
print(df_pivot.head(3))
df_pivot.to_csv(Location + 'population_analysis.csv', index=False)

Sex                     Female                                            \
Single Year of Age         0      1      2      3      4      5      6     
Administrative Counties                                                    
Carlow County Council    353.0  302.0  334.0  378.0  369.0  365.0  367.0   
Cavan County Council     501.0  477.0  520.0  525.0  492.0  544.0  591.0   
Clare County Council     691.0  704.0  744.0  745.0  766.0  799.0  789.0   

Sex                                           ...  Male                    \
Single Year of Age         7      8      9    ...   91    92    93    94    
Administrative Counties                       ...                           
Carlow County Council    455.0  404.0  435.0  ...  27.0  14.0   4.0   6.0   
Cavan County Council     555.0  615.0  602.0  ...  36.0  15.0  15.0  12.0   
Clare County Council     805.0  801.0  873.0  ...  49.0  28.0  20.0  14.0   

Sex                                                       
Single Year of Age   

In [128]:
df_pivot.columns.names

FrozenList(['Sex', 'Single Year of Age'])

In [129]:
# Get the sum of population for each counties female
female_counties_population = df_pivot['Female'].sum(axis=1)
female_counties_population.head(3)


Administrative Counties
Carlow County Council    31146.0
Cavan County Council     40644.0
Clare County Council     65021.0
dtype: float64

In [130]:
# Get the sum of population for each counties Male
male_counties_population = df_pivot['Male'].sum(axis=1)
male_counties_population.head(3)

Administrative Counties
Carlow County Council    30822.0
Cavan County Council     41060.0
Clare County Council     62917.0
dtype: float64

In [131]:
# Get the total population for Female age groups
female_age_population = df_pivot['Female'].sum(axis=0)
female_age_population.head(3)

Single Year of Age
0    28186.0
1    27545.0
2    28974.0
dtype: float64

In [132]:
# Get the total population for Male age groups
male_age_population = df_pivot['Male'].sum(axis=0)
male_age_population.head(3)

Single Year of Age
0    29610.0
1    28875.0
2    30236.0
dtype: float64

In [133]:
# Weighted mean calculation 
# calculation for weighted mean age (by sex)

ages = np.array(range(0, 101))  # Ages from 0 to 100

weighted_mean_female = np.average(ages, weights=female_age_population)
weighted_mean_male = np.average(ages, weights=male_age_population)
weighted_mean_female, weighted_mean_male

(np.float64(38.9397958987787), np.float64(37.7394477371039))

In [134]:
population_differance = []
for age in range(0, 101):
    if male_age_population.tolist()[age] > female_age_population.tolist()[age]:
        population_differance = male_age_population.tolist()[age] - female_age_population.tolist()[age]
        print(f"Male population is higher than the Female population by {population_differance} at age {age}.")
    else:
        population_differance = female_age_population.tolist()[age] - male_age_population.tolist()[age]
        print(f"Female population is higher than the Male population by {population_differance} at age {age}.")      

Male population is higher than the Female population by 1424.0 at age 0.
Male population is higher than the Female population by 1330.0 at age 1.
Male population is higher than the Female population by 1262.0 at age 2.
Male population is higher than the Female population by 1518.0 at age 3.
Male population is higher than the Female population by 1867.0 at age 4.
Male population is higher than the Female population by 1342.0 at age 5.
Male population is higher than the Female population by 1470.0 at age 6.
Male population is higher than the Female population by 1857.0 at age 7.
Male population is higher than the Female population by 1859.0 at age 8.
Male population is higher than the Female population by 1742.0 at age 9.
Male population is higher than the Female population by 1969.0 at age 10.
Male population is higher than the Female population by 1433.0 at age 11.
Male population is higher than the Female population by 1660.0 at age 12.
Male population is higher than the Female popula

In [135]:
# Get the age group number
age_group = 37

In [136]:
total_male_population_group = 0
total_female_population_group = 0
for age in range(age_group, age_group+5):
    total_male_population_group = male_age_population.tolist()[age] + total_male_population_group
    total_female_population_group = female_age_population.tolist()[age] + total_female_population_group

population_differance_age_group = total_male_population_group - total_female_population_group
print(f"Population differance between the age group {age_group} to {age_group+5} between male and female are {population_differance_age_group}")

Population differance between the age group 37 to 42 between male and female are -15135.0


# END